In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image

In [2]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

# Content Loss Layer
class ContentLoss(nn.Module):
	def __init__(self, target):
		super(ContentLoss, self).__init__()
		self.target = target.detach()

	def forward(self, input):
		self.loss = F.mse_loss(input, self.target)
		return input

# Style Loss Layer
class StyleLoss(nn.Module):
	def __init__(self, target_feature):
		super(StyleLoss, self).__init__()
		self.target = gram_matrix(target_feature).detach()

	def forward(self, input):
		G = gram_matrix(input)
		self.loss = F.mse_loss(G, self.target)
		return input

def gram_matrix(input):
	a, b, c, d = input.size()
	features = input.view(a* b, c*d)
	G = torch.mm(features, features.t())
	return G.div(a*b*c*d)

# Normalization Layer to transform input images
class Normalization(nn.Module):
	def __init__(self, mean, std):
		super(Normalization, self).__init__()
		self.mean = torch.tensor(mean).view(-1, 1, 1)
		self.std = torch.tensor(std).view(-1, 1, 1)

	def forward(self, image):
		return (image - self.mean) / self.std

# Create our model with our loss layers
def style_cnn(cnn, device, normalization_mean, normalization_std, style_image, content_image):
	# Insert loss layers after these desired layers
	style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
	content_layers = ['conv_4']
	
	# Copy network to work on
	cnn = copy.deepcopy(cnn)

	# Keep track of our losses
	style_losses = []
	content_losses = []

	# Start by normalizing our image
	normalization = Normalization(normalization_mean, normalization_std).to(device)
	model = nn.Sequential(normalization)

	# Keep track of convolutional layers
	i = 0

	# Loop through vgg layers
	for layer in cnn.children():
		if isinstance(layer, nn.Conv2d):
			i += 1
			name = 'conv_{}'.format(i)
		elif isinstance(layer, nn.ReLU):
			name = 'relu_{}'.format(i)
			layer = nn.ReLU(inplace=False)
		elif isinstance(layer, nn.MaxPool2d):
			name = 'pool_{}'.format(i)
		elif isinstance(layer, nn.BatchNorm2d):
			name = 'bn_{}'.format(i)

		# Add layer to our model
		model.add_module(name, layer)

		# Insert style loss layer
		if name in style_layers:
			target_feature = model(style_image).detach()
			style_loss = StyleLoss(target_feature)
			model.add_module('style_loss_{}'.format(i), style_loss)
			style_losses.append(style_loss)

		# Insert content loss layer
		if name in content_layers:
			target = model(content_image).detach()
			content_loss = ContentLoss(target)
			model.add_module('content_loss_{}'.format(i), content_loss)
			content_losses.append(content_loss)

	# Get rid of unneeded layers after our final losses
	for i in range(len(model) - 1, -1, -1):
		if isinstance(model[i], StyleLoss) or isinstance(model[i], ContentLoss):
			break

	model = model[:(i + 1)]

	return model, style_losses, content_losses

In [3]:
import os
import torch
from PIL import Image
import torchvision.transforms as T

# Set device (GPU if available, else CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Image size
SIZE = 512  

# Define image transformations
loader = T.Compose([
    T.Resize(SIZE),      
    T.CenterCrop(SIZE),  
    T.ToTensor()         
])

unloader = T.ToPILImage()  

# Function to Load and Process Image
def load_image(path):
    image = loader(Image.open(path).convert("RGB")).unsqueeze(0)  # Convert to RGB
    return image.to(DEVICE, torch.float)  

# Function to Save Processed Image
def save_image(tensor, path):
    image = unloader(tensor.cpu().clone().squeeze(0))  
    image.save(path)  

# Function to Process All Images in a Folder
def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):  
        os.makedirs(output_folder)  # Create output folder if it doesn’t exist

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Process only image files
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            print(f"Processing: {input_path} → {output_path}")
            img_tensor = load_image(input_path)  
            save_image(img_tensor, output_path)  

# Define folders
content_folder = "Q3/content"
styles_folder = "Q3/styles"
content_output_folder = "Q3/content2"
styles_output_folder = "Q3/styles2"

# Process content and styles images
process_images(content_folder, content_output_folder)
process_images(styles_folder, styles_output_folder)

print("Processing complete! ✅")

# to check if size of new image is 512x512 
img = Image.open("Q3/content2/cat.jpg")
width, height = img.size
print(f"Width: {width}, Height: {height}")


Using device: cuda
Processing: Q3/content/bear.jpg → Q3/content2/bear.jpg
Processing: Q3/content/building.jpg → Q3/content2/building.jpg
Processing: Q3/content/cat-on-table.jpg → Q3/content2/cat-on-table.jpg
Processing: Q3/content/cat-sleeping.jpg → Q3/content2/cat-sleeping.jpg
Processing: Q3/content/cat.jpg → Q3/content2/cat.jpg
Processing: Q3/content/cows.jpg → Q3/content2/cows.jpg
Processing: Q3/content/mountains.jpg → Q3/content2/mountains.jpg
Processing: Q3/content/picnic.jpg → Q3/content2/picnic.jpg
Processing: Q3/content/town.jpg → Q3/content2/town.jpg
Processing: Q3/content/white-building.jpg → Q3/content2/white-building.jpg
Processing: Q3/styles/bet-you.jpg → Q3/styles2/bet-you.jpg
Processing: Q3/styles/Erin-Hanson-Monet's-Bridge.jpg → Q3/styles2/Erin-Hanson-Monet's-Bridge.jpg
Processing: Q3/styles/head-of-paula-eyles.jpg → Q3/styles2/head-of-paula-eyles.jpg
Processing: Q3/styles/horse-cart.jpg → Q3/styles2/horse-cart.jpg
Processing: Q3/styles/landscape-with-a-palace.jpg → Q3/

In [4]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

# Content Loss Layer
class ContentLoss(nn.Module):
	def __init__(self, target):
		super(ContentLoss, self).__init__()
		self.target = target.detach()

	def forward(self, input):
		self.loss = F.mse_loss(input, self.target)
		return input

# Style Loss Layer
class StyleLoss(nn.Module):
	def __init__(self, target_feature):
		super(StyleLoss, self).__init__()
		self.target = gram_matrix(target_feature).detach()

	def forward(self, input):
		G = gram_matrix(input)
		self.loss = F.mse_loss(G, self.target)
		return input

def gram_matrix(input):
	a, b, c, d = input.size()
	features = input.view(a* b, c*d)
	G = torch.mm(features, features.t())
	return G.div(a*b*c*d)

# Normalization Layer to transform input images
class Normalization(nn.Module):
	def __init__(self, mean, std):
		super(Normalization, self).__init__()
		self.mean = torch.tensor(mean).view(-1, 1, 1)
		self.std = torch.tensor(std).view(-1, 1, 1)

	def forward(self, image):
		return (image - self.mean) / self.std

# Create our model with our loss layers
def style_cnn(cnn, device, normalization_mean, normalization_std, style_image, content_image):
	# Insert loss layers after these desired layers
	style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
	content_layers = ['conv_4']
	
	# Copy network to work on
	cnn = copy.deepcopy(cnn)

	# Keep track of our losses
	style_losses = []
	content_losses = []

	# Start by normalizing our image
	normalization = Normalization(normalization_mean, normalization_std).to(device)
	model = nn.Sequential(normalization)

	# Keep track of convolutional layers
	i = 0

	# Loop through vgg layers
	for layer in cnn.children():
		if isinstance(layer, nn.Conv2d):
			i += 1
			name = 'conv_{}'.format(i)
		elif isinstance(layer, nn.ReLU):
			name = 'relu_{}'.format(i)
			layer = nn.ReLU(inplace=False)
		elif isinstance(layer, nn.MaxPool2d):
			name = 'pool_{}'.format(i)
		elif isinstance(layer, nn.BatchNorm2d):
			name = 'bn_{}'.format(i)

		# Add layer to our model
		model.add_module(name, layer)

		# Insert style loss layer
		if name in style_layers:
			target_feature = model(style_image).detach()
			style_loss = StyleLoss(target_feature)
			model.add_module('style_loss_{}'.format(i), style_loss)
			style_losses.append(style_loss)

		# Insert content loss layer
		if name in content_layers:
			target = model(content_image).detach()
			content_loss = ContentLoss(target)
			model.add_module('content_loss_{}'.format(i), content_loss)
			content_losses.append(content_loss)

	# Get rid of unneeded layers after our final losses
	for i in range(len(model) - 1, -1, -1):
		if isinstance(model[i], StyleLoss) or isinstance(model[i], ContentLoss):
			break

	model = model[:(i + 1)]

	return model, style_losses, content_losses

In [5]:
import os
import torch
import torchvision.models as models
import torch.optim as optim
from PIL import Image

# Constants
EPOCHS = 200
STYLE_WEIGHTS = [1000, 1000, 1000, 1000, 1000]
CONTENT_WEIGHTS = [1, 5, 10, 20, 50]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
STYLE_FOLDER = "Q3/styles2"
CONTENT_FOLDER = "Q3/content2"
OUTPUT_FOLDER = "output"

# Ensure output directory exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Load all style and content images
style_files = sorted([f for f in os.listdir(STYLE_FOLDER) if f.endswith('.jpg')])
content_files = sorted([f for f in os.listdir(CONTENT_FOLDER) if f.endswith('.jpg')])

# Load VGG model once
cnn = models.vgg19(pretrained=True).features.to(DEVICE).eval()
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(DEVICE)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(DEVICE)

def process_image_pair(style_path, content_path, output_prefix, optimizer_type, style_weight, content_weight):
    print(f"Processing {os.path.basename(content_path)} with {os.path.basename(style_path)} using {optimizer_type}, Style Weight: {style_weight}, Content Weight: {content_weight}")
    
    # Load images
    style_image = load_image(style_path)
    content_image = load_image(content_path)
    
    # Initialize target image
    target_image = content_image.clone()
    
    # Build style transfer model
    model, style_losses, content_losses = style_cnn(cnn, DEVICE, 
        cnn_normalization_mean, cnn_normalization_std, style_image, content_image)
    
    # Define optimizer
    if optimizer_type == 'LBFGS':
        optimizer = optim.LBFGS([target_image.requires_grad_()])
    else:  # Adam
        optimizer = optim.Adam([target_image.requires_grad_()], lr=0.01)
    
    run = [0]
    def closure():
        target_image.data.clamp_(0, 1)
        optimizer.zero_grad()
        model(target_image)
        style_score = sum(s.loss for s in style_losses)
        content_score = sum(c.loss for c in content_losses)
        loss = (style_score * style_weight) + (content_score * content_weight)
        loss.backward()
        run[0] += 1
        if run[0] % 200 == 0:
            print(f"Run {run[0]} - Style Loss: {style_score.item():.4f} Content Loss: {content_score.item():.4f}")
        return loss
    
    if optimizer_type == 'LBFGS':
        while run[0] < EPOCHS:
            optimizer.step(closure)
    else:  # Adam
        for _ in range(EPOCHS):
            optimizer.step(closure)
    
    # Save result
    output_path = os.path.join(OUTPUT_FOLDER, f"{output_prefix}_{optimizer_type}_SW{style_weight}_CW{content_weight}.jpg")
    save_image(target_image, output_path)
    print(f"Saved output: {output_path}\n")
    
    # Free memory
    del style_image, content_image, target_image, model, style_losses, content_losses
    torch.cuda.empty_cache()

# Process first 5 images with both optimizers and different weight values
for i in range(min(5, len(style_files), len(content_files))):
    style_path = os.path.join(STYLE_FOLDER, style_files[i])
    content_path = os.path.join(CONTENT_FOLDER, content_files[i])
    
    for style_weight, content_weight in zip(STYLE_WEIGHTS, CONTENT_WEIGHTS):
        process_image_pair(style_path, content_path, f"output_{i+1}", 'LBFGS', style_weight, content_weight)
        torch.cuda.empty_cache()
        process_image_pair(style_path, content_path, f"output_{i+1}", 'Adam', style_weight, content_weight)
        torch.cuda.empty_cache()

print("✅ All images processed with both optimizers for different weight values!")


/home/sid/miniconda3/envs/md-intro-tutorial/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sid/miniconda3/envs/md-intro-tutorial/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing bear.jpg with Erin-Hanson-Monet's-Bridge.jpg using LBFGS, Style Weight: 1000, Content Weight: 1


/tmp/ipykernel_29564/4032233483.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean).view(-1, 1, 1)
/tmp/ipykernel_29564/4032233483.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std).view(-1, 1, 1)


Run 200 - Style Loss: 19.1936 Content Loss: 424.3852
Saved output: output/output_1_LBFGS_SW1000_CW1.jpg

Processing bear.jpg with Erin-Hanson-Monet's-Bridge.jpg using Adam, Style Weight: 1000, Content Weight: 1
Run 200 - Style Loss: 0.0033 Content Loss: 7.3786
Saved output: output/output_1_Adam_SW1000_CW1.jpg

Processing bear.jpg with Erin-Hanson-Monet's-Bridge.jpg using LBFGS, Style Weight: 1000, Content Weight: 5
Run 200 - Style Loss: 0.0143 Content Loss: 2.6860
Saved output: output/output_1_LBFGS_SW1000_CW5.jpg

Processing bear.jpg with Erin-Hanson-Monet's-Bridge.jpg using Adam, Style Weight: 1000, Content Weight: 5
Run 200 - Style Loss: 0.0161 Content Loss: 2.2823
Saved output: output/output_1_Adam_SW1000_CW5.jpg

Processing bear.jpg with Erin-Hanson-Monet's-Bridge.jpg using LBFGS, Style Weight: 1000, Content Weight: 10
Run 200 - Style Loss: 0.0282 Content Loss: 0.6051
Saved output: output/output_1_LBFGS_SW1000_CW10.jpg

Processing bear.jpg with Erin-Hanson-Monet's-Bridge.jpg using

# Comparison of Adam and L-BFGS for Style Transfer

## Observations

### Style Loss:
- Adam and L-BFGS resulted in nearly identical style losses across different content weights.
- At lower content weights (CW=1), L-BFGS had slightly lower style loss in some cases.
- At higher content weights (CW=50), both optimizers achieved near-identical style losses.

### Content Loss:
- L-BFGS consistently produced slightly lower content loss than Adam.
- The gap is more noticeable at lower content weights but diminishes as CW increases.
- The trend suggests that L-BFGS preserves content details better.

### Effect of Content Weight:
- As CW increases, content loss decreases significantly.
- At CW=50, both optimizers achieve very low content loss (~0.001–0.025), indicating near-perfect content preservation.

## Key Takeaways
- **L-BFGS slightly outperforms Adam in preserving content, especially at lower content weights.**
- **Both optimizers achieve similar style loss at higher CW, but L-BFGS achieves slightly lower content loss overall.**
- **Adam may be preferable for consistency and computational efficiency, while L-BFGS may be better for maximizing content preservation.**

### Outputs stored in outputs dir